# Análisis de accidentes con Spark

En este laboratorio lo que se busca es poder encotrar información interesante y relevante con respecto a los accidentes que han ocurrido de los años de 2013 a 2023 en Guatemala. Principalmente tenemos 3 tipos de archivos: Fallecidos y Lesionados, Hechos de tránsito y Vehículos involucrados. Estos 3 archivos contienen información con respecto a accidentes automovílisticos. La idea es poder usar spark para poder analizar los eventos y las características relacionadas a este. Lo primero es juntar todos los archivos. Obtuvimos los datos de la página del Instituto Nacional de Estadística (INE): https://www.ine.gob.gt/bases-de-datos/accidentes-de-transito/. Descargando todos los archivos desde 2013 a 2023. El procesamiento se hizo en distintas etapas:

1. Descargar todos los datos en un sav y si no esta disponible en ese formato, en excel.
2. Juntar todos los dataframes en los 3 tipos de archivos descritos (fallecidos y lesionados, hechos de tránsito y vehículos involucrados)
3. Analizar los datos y ver columnas con muchos valores vacíos, columnas repetidas entre otras situaciones que generen data innecesaria o redundante.


# Merge detodos los datos

In [12]:
import pandas as pd
import pyreadstat
import os
from pathlib import Path
import re

def detectar_formato_archivos(directorio):
    """Detecta los formatos de archivo disponibles por tipo"""
    tipos = {
        'hechos_transito': [],
        'vehiculos_involucrados': [],
        'fallecidos_lesionados': []
    }
    
    for archivo in Path(directorio).iterdir():
        # Manejar extensiones dobles como .sav.xlsx
        if archivo.suffix in ['.sav', '.xlsx'] or '.sav.xlsx' in archivo.name:
            nombre = archivo.stem.lower()
            # Remover .sav del nombre si existe extensión doble
            nombre = nombre.replace('.sav', '')
            
            if 'hechos_transito' in nombre:
                tipos['hechos_transito'].append(archivo)
            elif 'vehiculos_involucrados' in nombre:
                tipos['vehiculos_involucrados'].append(archivo)
            elif 'fallecidos_lesionados' in nombre:
                tipos['fallecidos_lesionados'].append(archivo)
    
    return tipos

def determinar_formato_archivo(archivo):
    """Determina el formato real del archivo"""
    nombre = archivo.name.lower()
    
    if nombre.endswith('.sav'):
        return 'spss'
    elif nombre.endswith('.xlsx') or nombre.endswith('.sav.xlsx'):
        return 'excel'
    else:
        return 'desconocido'

def leer_archivo(archivo):
    """Lee un archivo según su formato (SPSS o Excel)"""
    try:
        formato = determinar_formato_archivo(archivo)
        
        if formato == 'spss':
            df, meta = pyreadstat.read_sav(archivo)
            print(f"   {archivo.name}: {len(df)} registros, {len(df.columns)} columnas (SPSS)")
            return df
        elif formato == 'excel':
            df = pd.read_excel(archivo)
            print(f"   {archivo.name}: {len(df)} registros, {len(df.columns)} columnas (Excel)")
            return df
        else:
            print(f"   Formato no soportado: {archivo.name}")
            return None
            
    except Exception as e:
        print(f"   Error leyendo {archivo.name}: {e}")
        return None

def extraer_anio(nombre_archivo):
    """Extrae el año del nombre del archivo"""
    # Primero limpiar el nombre de extensiones dobles
    nombre_limpio = nombre_archivo.stem.replace('.sav', '').replace('.xlsx', '')
    match = re.search(r'(\d{4})', nombre_limpio)
    return match.group(1) if match else "Desconocido"

def unir_archivos_por_tipo(lista_archivos, tipo):
    """Une archivos del mismo tipo manejando discrepancias de columnas"""
    print(f"\n Procesando archivos de {tipo}:")
    
    dataframes = []
    problemas = []
    
    for archivo in sorted(lista_archivos):
        año = extraer_anio(archivo)
        print(f"  Año {año}: {archivo.name}")
        
        df = leer_archivo(archivo)
        if df is not None:
            # Agregar metadatos
            df['año'] = año
            df['archivo_origen'] = archivo.name
            df['tipo_dataset'] = tipo
            
            dataframes.append(df)
        else:
            problemas.append(archivo.name)
    
    if dataframes:
        # Unir todos los DataFrames manejando columnas diferentes
        df_final = pd.concat(dataframes, ignore_index=True, sort=False)
        
        print(f"\n {tipo} - Resultado:")
        print(f"   - Total registros: {len(df_final)}")
        print(f"   - Total columnas: {len(df_final.columns)}")
        print(f"   - Años incluidos: {sorted(df_final['año'].unique())}")
        
        return df_final
    else:
        print(f" No se pudo procesar ningún archivo de {tipo}")
        return None

def analizar_estructura_columnas(dfs_unidos):
    """Analiza las columnas comunes y diferentes entre datasets"""
    print("\n" + "="*60)
    print(" ANÁLISIS DE ESTRUCTURA DE COLUMNAS")
    print("="*60)
    
    for tipo, df in dfs_unidos.items():
        if df is not None:
            print(f"\n{tipo.upper()}:")
            print(f"  Columnas ({len(df.columns)}): {list(df.columns)}")
            
            # Verificar valores nulos por columna
            print(f"  Valores nulos por columna:")
            for col in df.columns:
                nulos = df[col].isna().sum()
                if nulos > 0:
                    print(f"    - {col}: {nulos} nulos ({nulos/len(df)*100:.1f}%)")


# Configuración
directorio_datos = "./data"
directorio_salida = "./datasets_unidos"

print(" Viendo archivos...")
tipos_archivos = detectar_formato_archivos(directorio_datos)

# Mostrar archivos encontrados
for tipo, archivos in tipos_archivos.items():
    print(f"\n{tipo}:")
    for archivo in sorted(archivos):
        formato = determinar_formato_archivo(archivo)
        print(f"  - {archivo.name} [{formato.upper()}]")

# Unir archivos por tipo
dfs_unidos = {}
for tipo, archivos in tipos_archivos.items():
    if archivos:
        dfs_unidos[tipo] = unir_archivos_por_tipo(archivos, tipo)
    else:
        print(f"\n  No se encontraron archivos para {tipo}")
        dfs_unidos[tipo] = None

# Guardar resultados
print(f"\n Guardando datasets unidos...")
Path(directorio_salida).mkdir(exist_ok=True)

for tipo, df in dfs_unidos.items():
    if df is not None:
        archivo_salida = Path(directorio_salida) / f"{tipo}_completo.csv"
        df.to_csv(archivo_salida, index=False, encoding='utf-8-sig')
        print(f" {tipo} guardado en: {archivo_salida}")

# Resumen final
print(f"\n PROCESO COMPLETADO")
print("="*40)
for tipo, df in dfs_unidos.items():
    if df is not None:
        print(f"{tipo}: {len(df):,} registros, {len(df.columns)} columnas")
        print(f"     Años: {sorted(df['año'].unique())}")
    else:
        print(f"{tipo}: No procesado")



 Viendo archivos...

hechos_transito:
  - hechos_transito2013.sav [SPSS]
  - hechos_transito2014.sav [SPSS]
  - hechos_transito2015.xlsx [EXCEL]
  - hechos_transito2016.sav [SPSS]
  - hechos_transito2017.sav [SPSS]
  - hechos_transito2018.sav [SPSS]
  - hechos_transito2019.sav [SPSS]
  - hechos_transito2020.sav [SPSS]
  - hechos_transito2021.sav [SPSS]
  - hechos_transito2022.sav [SPSS]
  - hechos_transito2023.sav [SPSS]

vehiculos_involucrados:
  - vehiculos_involucrados2013.sav [SPSS]
  - vehiculos_involucrados2014.sav [SPSS]
  - vehiculos_involucrados2015.xlsx [EXCEL]
  - vehiculos_involucrados2016.sav [SPSS]
  - vehiculos_involucrados2017.sav [SPSS]
  - vehiculos_involucrados2018.sav [SPSS]
  - vehiculos_involucrados2019.sav [SPSS]
  - vehiculos_involucrados2020.sav [SPSS]
  - vehiculos_involucrados2021.sav [SPSS]
  - vehiculos_involucrados2022.sav [SPSS]
  - vehiculos_involucrados2023.sav [SPSS]

fallecidos_lesionados:
  - fallecidos_lesionados2013.sav [SPSS]
  - fallecidos_lesion

se puede ver que se han logrado juntar la información de todos los años, incluso teniendo archivos de distinto tipo. Ahora es encesario evaluar las columnas y ver cuales podrían no ser necesaríass o incluso problemáticas.

In [13]:
analizar_estructura_columnas(dfs_unidos)


 ANÁLISIS DE ESTRUCTURA DE COLUMNAS

HECHOS_TRANSITO:
  Columnas (49): ['num_hecho', 'dia_ocu', 'mes_ocu', 'dia_sem_ocu', 'hora_ocu', 'g_hora', 'depto_ocu', 'mupio_ocu', 'areag_ocu', 'zona_ocu', 'sexo_pil', 'edad_pil', 'g_edad_2', 'mayor_menor', 'tipo_veh', 'color_veh', 'modelo_veh', 'causa_acc', 'marca_veh', 'estado_pil', 'año', 'archivo_origen', 'tipo_dataset', 'num_correlativo', 'corre_base', 'día_ocu', 'día_sem_ocu', 'área_geo_ocu', 'sexo_con', 'edad_con', 'g_edad', 'estado_con', 'tipo_eve', 'núm_corre', 'año_ocu', 'g_hora_5', 'sexo_per', 'edad_per', 'g_edad_80ymás', 'g_edad_60ymás', 'edad_quinquenales', 'g_modelo_veh', 'Núm_corre', 'Año_ocu', 'Día_ocu', 'Hora_ocu', 'Mes_ocu', 'zona_ciudad', 'num_corre']
  Valores nulos por columna:
    - num_hecho: 70435 nulos (91.8%)
    - dia_ocu: 70435 nulos (91.8%)
    - mes_ocu: 6350 nulos (8.3%)
    - dia_sem_ocu: 70435 nulos (91.8%)
    - hora_ocu: 6350 nulos (8.3%)
    - areag_ocu: 70435 nulos (91.8%)
    - sexo_pil: 70435 nulos (91.8%)
 

Viendo estos resultados es claro que hay algunos problemas, tenemos más de 40 columnnas en los 3 archivos y varias de esas columnas tienen valores nulos. Es más hay algunas columnas que son literalmente lo mismo, pero debido a ortografía (espacios, tildes, diferencias en el nombre) hay columnas que se marcan como diferente. Sabiendo esto es necesario limitar la selección de datos y jutnar la información redundante.

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
import unicodedata
import re

def estandarizar_nombres_columnas(df):
    """Estandariza nombres de columnas: minusculas, sin tildes, sin espacios"""
    def limpiar_nombre(col):
        # Convertir a string y minusculas
        col = str(col).lower()
        # Remover tildes
        col = unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('ASCII')
        # Reemplazar espacios y caracteres especiales
        col = col.replace(' ', '_').replace('-', '_').replace('.', '_')
        return col
    
    new_columns = [limpiar_nombre(col) for col in df.columns]
    df.columns = new_columns
    return df

def eliminar_columnas_duplicadas(df):
    """Elimina columnas duplicadas por nombre"""
    print("   Eliminando columnas duplicadas...")
    
    # Identificar columnas duplicadas
    columnas_duplicadas = df.columns[df.columns.duplicated()].tolist()
    
    if columnas_duplicadas:
        print(f"     Encontradas {len(columnas_duplicadas)} columnas duplicadas: {columnas_duplicadas}")
        # Mantener solo la primera ocurrencia de cada columna
        df = df.loc[:, ~df.columns.duplicated()]
        print(f"     Columnas duplicadas eliminadas")
    
    return df

def extraer_ano_archivo(archivo_str):
    """Extrae el año del string del archivo"""
    # Buscar un patrón de 4 dígitos (año)
    match = re.search(r'(\d{4})', archivo_str)
    if match:
        return match.group(1)
    else:
        return None

def combinar_columnas_similares(df):
    """
    Combina columnas similares para reducir valores nulos.
    Asume que los nombres ya están estandarizados.
    """
    print("   Combinando columnas similares...")
    
    # Mapeo de columnas a consolidar. La clave es el nombre de la columna objetivo.
    mapeo = {
        'num_hecho': ['num_hecho', 'num_correlativo', 'num_corre'],
        'dia_ocu': ['dia_ocu'],
        'mes_ocu': ['mes_ocu'],
        'hora_ocu': ['hora_ocu'],
        'dia_sem_ocu': ['dia_sem_ocu'],
        'depto_ocu': ['depto_ocu'],
        'mupio_ocu': ['mupio_ocu'],
        'areag_ocu': ['areag_ocu', 'area_geo_ocu'],
        'zona_ocu': ['zona_ocu', 'zona_ciudad'],
        'sexo_pil': ['sexo_pil', 'sexo_con', 'sexo_per'],
        'edad_pil': ['edad_pil', 'edad_con', 'edad_per'],
        'tipo_veh': ['tipo_veh'],
        'color_veh': ['color_veh'],
        'modelo_veh': ['modelo_veh'],
        'causa_acc': ['causa_acc'],
        'marca_veh': ['marca_veh'],
        'estado_pil': ['estado_pil', 'estado_con'],
        'fallecidos_lesionados': ['fallecidos_lesionados', 'fall_les'],
        'tipo_eve': ['tipo_eve']
    }
    
    for col_principal, columnas in mapeo.items():
        # Filtrar las columnas que existen en el DataFrame
        columnas_existentes = [col for col in columnas if col in df.columns]
        if len(columnas_existentes) > 1:
            print(f"     Combinando {columnas_existentes} en {col_principal}")
            # Si la columna principal no está en el DataFrame, usamos la primera de las existentes
            if col_principal not in df.columns:
                col_principal = columnas_existentes[0]
                # Si la columna principal no es la primera, la renombramos
                if col_principal != columnas_existentes[0]:
                    df = df.rename(columns={col_principal: columnas_existentes[0]})
                    col_principal = columnas_existentes[0]
            # Para cada columna similar (excluyendo la principal)
            for col in columnas_existentes:
                if col != col_principal:
                    # Combinar: donde la principal tiene nulos y la similar tiene valor, copiamos el valor.
                    mask = df[col_principal].isna() & df[col].notna()
                    df.loc[mask, col_principal] = df.loc[mask, col]
                    # Eliminamos la columna similar
                    df = df.drop(columns=[col])
    
    return df

def limpiar_y_seleccionar_columnas(df, tipo_dataset):
    """Funcion generica para limpiar y seleccionar columnas relevantes"""
    print(f"Limpiando dataset: {tipo_dataset.upper()}")
    
    # Definir columnas relevantes segun el tipo de dataset
    if tipo_dataset == 'hechos_transito':
        columnas_relevantes = [
            'num_hecho', 'dia_ocu', 'mes_ocu', 'dia_sem_ocu', 'hora_ocu',
            'depto_ocu', 'mupio_ocu', 'areag_ocu', 'zona_ocu', 
            'tipo_veh', 'color_veh', 'modelo_veh', 'causa_acc',
            'sexo_pil', 'edad_pil'
        ]
    elif tipo_dataset == 'vehiculos_involucrados':
        columnas_relevantes = [
            'num_hecho', 'dia_ocu', 'mes_ocu', 'hora_ocu',
            'depto_ocu', 'mupio_ocu', 'areag_ocu', 'zona_ocu',
            'tipo_veh', 'color_veh', 'modelo_veh', 'marca_veh',
            'sexo_pil', 'edad_pil', 'estado_pil', 'causa_acc'
        ]
    elif tipo_dataset == 'fallecidos_lesionados':
        columnas_relevantes = [
            'num_hecho', 'dia_ocu', 'mes_ocu', 'hora_ocu',
            'depto_ocu', 'mupio_ocu', 'areag_ocu', 'zona_ocu',
            'tipo_veh', 'causa_acc', 'sexo_pil', 'edad_pil',
            'fallecidos_lesionados', 'tipo_eve'
        ]
    else:
        columnas_relevantes = []
    
    # Agregar columnas de metadatos
    columnas_metadatos = ['archivo_origen', 'tipo_dataset']
    
    # Seleccionar solo columnas existentes
    columnas_existentes = [col for col in df.columns if col in columnas_relevantes + columnas_metadatos]
    
    # Verificar que tenemos columnas minimas
    columnas_minimas = ['depto_ocu']  # Ya no requerimos 'año' aquí porque lo extraeremos después
    columnas_faltantes = [col for col in columnas_minimas if col not in columnas_existentes]
    if columnas_faltantes:
        print(f"   Advertencia: Faltan columnas minimas {columnas_faltantes} en {tipo_dataset}")
    
    # Si no hay columnas relevantes, usar todas las disponibles
    if not columnas_existentes:
        print(f"   No se encontraron columnas relevantes, usando todas las columnas disponibles")
        columnas_existentes = list(dict.fromkeys(df.columns.tolist()))  # Eliminar duplicados
    
    # Crear nuevo DataFrame con columnas seleccionadas
    df_limpio = df[columnas_existentes].copy()
    
    # Extraer el año desde 'archivo_origen' y crear la columna 'año'
    if 'archivo_origen' in df_limpio.columns:
        df_limpio['año'] = df_limpio['archivo_origen'].apply(extraer_ano_archivo)
        print(f"   - Columna 'año' extraída de 'archivo_origen'")
    
    # Limpiar datos
    registros_originales = len(df_limpio)
    df_limpio = df_limpio.drop_duplicates().reset_index(drop=True)
    
    print(f"   - Columnas originales: {len(df.columns)}")
    print(f"   - Columnas finales: {len(df_limpio.columns)}")
    print(f"   - Registros originales: {registros_originales}")
    print(f"   - Registros despues de eliminar duplicados: {len(df_limpio)}")
    print(f"   - Columnas seleccionadas: {list(df_limpio.columns)}")
    
    return df_limpio

def analizar_calidad_datos(df, nombre_dataset):
    """Analiza la calidad de los datos despues de la limpieza"""
    if df is None or len(df) == 0:
        print(f"ERROR: {nombre_dataset}: Dataset vacio o no procesado")
        return
    
    print(f"CALIDAD DE DATOS - {nombre_dataset}:")
    print(f"   - Total registros: {len(df):,}")
    print(f"   - Total columnas: {len(df.columns)}")
    
    if 'año' in df.columns:
        print(f"   - Años disponibles: {sorted(df['año'].unique())}")
    
    print(f"   - Valores nulos por columna:")
    for col in df.columns:
        try:
            # Asegurarnos de que estamos trabajando con una sola columna
            if col in df.columns:
                nulos = df[col].isna().sum()
                # Verificar si nulos es un escalar (numero) o una Serie
                if hasattr(nulos, 'item'):  # Si es una Serie/array
                    nulos = nulos.item()
                
                if pd.isna(nulos):
                    nulos = 0
                
                if nulos > 0:
                    porcentaje = (nulos / len(df)) * 100
                    print(f"     - {col}: {nulos:,} nulos ({porcentaje:.1f}%)")
        except Exception as e:
            print(f"     - {col}: Error calculando nulos - {e}")

# Configuracion
directorio_entrada = "./datasets_unidos"
directorio_salida = "./datasets_limpios"

Path(directorio_salida).mkdir(exist_ok=True)

try:
    # Cargar datasets unidos
    print("Cargando datasets unidos...")
    
    hechos = pd.read_csv(f"{directorio_entrada}/hechos_transito_completo.csv")
    vehiculos = pd.read_csv(f"{directorio_entrada}/vehiculos_involucrados_completo.csv")
    fallecidos = pd.read_csv(f"{directorio_entrada}/fallecidos_lesionados_completo.csv")
    
    print(f"Datasets cargados:")
    print(f"   - Hechos: {len(hechos):,} registros, {len(hechos.columns)} columnas")
    print(f"   - Vehiculos: {len(vehiculos):,} registros, {len(vehiculos.columns)} columnas")
    print(f"   - Fallecidos: {len(fallecidos):,} registros, {len(fallecidos.columns)} columnas")
    
except Exception as e:
    print(f"Error cargando datasets: {e}")
    exit()

# Estandarizar nombres de columnas
print("\nEstandarizando nombres de columnas...")
hechos = estandarizar_nombres_columnas(hechos)
vehiculos = estandarizar_nombres_columnas(vehiculos)
fallecidos = estandarizar_nombres_columnas(fallecidos)

print("   Nombres de columnas estandarizados")

# Eliminar columnas duplicadas
print("\nEliminando columnas duplicadas...")
hechos = eliminar_columnas_duplicadas(hechos)
vehiculos = eliminar_columnas_duplicadas(vehiculos)
fallecidos = eliminar_columnas_duplicadas(fallecidos)

# Combinar columnas similares
print("\nCombinando columnas similares...")
hechos = combinar_columnas_similares(hechos)
vehiculos = combinar_columnas_similares(vehiculos)
fallecidos = combinar_columnas_similares(fallecidos)

# Limpiar datasets
print("\nLimpiando y seleccionando columnas relevantes...")
hechos_limpio = limpiar_y_seleccionar_columnas(hechos, 'hechos_transito')
vehiculos_limpio = limpiar_y_seleccionar_columnas(vehiculos, 'vehiculos_involucrados')
fallecidos_limpio = limpiar_y_seleccionar_columnas(fallecidos, 'fallecidos_lesionados')

# Analizar calidad
print("\n" + "="*60)
print("ANALISIS DE CALIDAD POST-LIMPIEZA")
print("="*60)

analizar_calidad_datos(hechos_limpio, "HECHOS_TRANSITO")
analizar_calidad_datos(vehiculos_limpio, "VEHICULOS_INVOLUCRADOS")
analizar_calidad_datos(fallecidos_limpio, "FALLECIDOS_LESIONADOS")

# Guardar datasets limpios
print(f"\nGuardando datasets limpios...")
try:
    hechos_limpio.to_csv(f"{directorio_salida}/hechos_transito_limpio.csv", index=False, encoding='utf-8-sig')
    vehiculos_limpio.to_csv(f"{directorio_salida}/vehiculos_involucrados_limpio.csv", index=False, encoding='utf-8-sig')
    fallecidos_limpio.to_csv(f"{directorio_salida}/fallecidos_lesionados_limpio.csv", index=False, encoding='utf-8-sig')
    print(f"Datasets limpios guardados en: {directorio_salida}")
except Exception as e:
    print(f"Error guardando datasets: {e}")

# Mostrar estructura final
print(f"\nESTRUCTURA FINAL:")
if hechos_limpio is not None:
    print(f"   Hechos transito: {list(hechos_limpio.columns)}")
if vehiculos_limpio is not None:
    print(f"   Vehiculos: {list(vehiculos_limpio.columns)}")
if fallecidos_limpio is not None:
    print(f"   Fallecidos/Lesionados: {list(fallecidos_limpio.columns)}")

Cargando datasets unidos...
Datasets cargados:
   - Hechos: 76,759 registros, 49 columnas
   - Vehiculos: 111,988 registros, 49 columnas
   - Fallecidos: 109,417 registros, 48 columnas

Estandarizando nombres de columnas...
   Nombres de columnas estandarizados

Eliminando columnas duplicadas...
   Eliminando columnas duplicadas...
     Encontradas 8 columnas duplicadas: ['dia_ocu', 'dia_sem_ocu', 'num_corre', 'ano_ocu', 'dia_ocu', 'hora_ocu', 'mes_ocu', 'num_corre']
     Columnas duplicadas eliminadas
   Eliminando columnas duplicadas...
     Encontradas 8 columnas duplicadas: ['dia_ocu', 'dia_sem_ocu', 'num_corre', 'num_corre', 'ano_ocu', 'dia_ocu', 'hora_ocu', 'mes_ocu']
     Columnas duplicadas eliminadas
   Eliminando columnas duplicadas...
     Encontradas 4 columnas duplicadas: ['dia_ocu', 'dia_sem_ocu', 'num_corre', 'num_corre']
     Columnas duplicadas eliminadas

Combinando columnas similares...
   Combinando columnas similares...
     Combinando ['num_hecho', 'num_correlativ

Luego de hacer varias pruebas este fue el mejor resultado para el data set. Es importante destacar que no todas las columnas fueron procesadas exitosamene, se a reducido la dimensión de los sets de datos. Esto pasó principalmente porque entre los distintos años habían varias columnas que o bien tenían nombres similares o columnas que solo se encontraban en algunos años en específico. Realmente el set de datos no tiene un id que relacione a las tres aparte del numero de correlativo o numero de hecho. Algunas de las estrategias que hicimos fue juntar los datos de columnas duplicadas y también columnas que significaban prácticamente lo mismo. Aún después de esto hay algunas columnas con valores núlos, pero al menos la mayoría parece estar funcionando y debería de ser suficiente para poder analizar los 3 archivos por separados y en conjunto. Para hacer los joins se espera poder usar cosas como llaves compuetas con el año, fecha, dia,zona y demás como sugiere la guía. 